In [6]:
import torch
import os
import json
from tqdm import tqdm
import shortuuid
import sys
from torch import nn
import sklearn
import pathlib
import pickle
import numpy as np
import torch.nn.functional as F
from scipy.special import rel_entr
from textblob import TextBlob    
import cv2
import re
from collections import OrderedDict
        
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../utils")))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../llava")))

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, process_images, KeywordsStoppingCriteria
from PIL import Image
import math
import matplotlib.pyplot as plt
from scipy.stats import entropy

from transformers import set_seed
from notebooks.utils.train_utils import get_answer_with_probability, calculate_distance
from notebooks.utils.diffusion_noise import add_diffusion_noise
import nltk

In [7]:
model_name = "liuhaotian/llava-v1.5-7b"
conv_mode='llava_v1'
cuda_device='3'
cd_beta = 0.1
cd_alpha = 1
temperature = 1.0
top_p = 1
top_k = None
device = torch.device("cuda:" + str(cuda_device) if torch.cuda.is_available() else "cpu")
model_path = os.path.expanduser(model_name)
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, None, model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.37s/it]


In [17]:
from PIL import Image
from collections import defaultdict, Counter
from evaluate import load
import ast
import re
from notebooks.utils.train_utils import get_answer_with_probability, calculate_distance, get_answer_with_probability_beams
from notebooks.utils.LAVE.eval_utils import lave_scorer
N = 0
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

masking_type = 'Diffusion'
noise_step = 900

# Save the generation if post processing needed
pathlib.Path().mkdir(parents=True, exist_ok=True)
answers_file = '/vault/erum/generations/llava7b_aokvqa_generation.json'
os.makedirs(os.path.dirname(answers_file), exist_ok=True)
# ans_file = open(answers_file, "w")
sequences = []
vcd_confidence_scores = []
noise_confidence_scores =  []
accQA = []
acc = 0.0

#AOKVQA
data_dir = '/vault/erum/AOKVQA'
attn_dir =  '/vault/erum/AOKVQA/attn_val2017/'
images_dir = '/vault/erum/AOKVQA/val2017/'
split = 'val'
data_file = os.path.join(data_dir, f"aokvqa_v1p0_{split}.json")
data = json.load(open(data_file))
data = [d for d in data if d['difficult_direct_answer'] is False]
image_filenames = os.listdir(images_dir)
imageid2filename = {}
for fn in image_filenames:
    image_id = int(fn.split('_')[-1].strip('.jpg'))
    imageid2filename[image_id] = os.path.join(images_dir, fn)
imageids = list(set(list(imageid2filename.keys())))


print(len(data))
directvqa_rollouts = defaultdict(dict)

N = 0
for i, datum in tqdm(enumerate(data)):   
    save_regdistributiontop5 = []
    save_augdistributiontop5 = []
    save_regdistributiontop2 = []
    save_augdistributiontop2 = []
    N += 1
    topk_distance = []

    image_filename = imageid2filename[datum['image_id']]
    qs = datum['question']
    answers_dict = datum['direct_answers']
    image_file = Image.open(image_filename).convert('RGB')

    answer_counter = Counter(answers_dict)

    score_dict = defaultdict(float)
    for a, c in answer_counter.items():
        score_dict[a] = min(1, c/3.0)

    augmented_rephrase_list = []
    reg_rephrase_list = []
    logits_aug2 = []
    
    cur_prompt = datum['question']
    prompt = f"USER: <image>\nAnswer the question using a single word or phrase: {cur_prompt} ASSISTANT:"
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device=device, non_blocking=True)
    image_tensor = process_images([image_file], image_processor, model.config)[0]
        
    
    if masking_type == 'Diffusion': 
        image_tensor_cd = add_diffusion_noise(image_tensor, noise_step)
    elif masking_type == 'attention':
        attn_filename = os.path.join(attn_dir, str(datum['image_id'])+'.jpg')
        image_file_attn = Image.open(attn_filename).convert('RGB')
        image_tensor_cd = process_images([image_file_attn], image_processor, model.config)[0]
    elif masking_type == 'black':
        image_tensor_cd = Image.open('black.png').convert('RGB')
        image_tensor_cd = image_tensor_cd.convert('RGB')
        image_tensor_cd = process_images([image_tensor_cd], image_processor, model.config)[0]
    else:
        image_tensor_cd = None

    image_tensor_cd2 = Image.open('black.png').convert('RGB')
    image_tensor_cd2 = image_tensor_cd2.convert('RGB')
    image_tensor_cd2 = process_images([image_tensor_cd2], image_processor, model.config)[0]

    attn_filename = os.path.join(attn_dir, str(datum['image_id'])+'.jpg')
    image_file_attn = Image.open(attn_filename).convert('RGB')
    image_tensor_cd3 = process_images([image_file_attn], image_processor, model.config)[0]

    # Black Image forward Pass
    with torch.inference_mode():
        outputs_aug2 = model.generate(input_ids, 
        images=image_tensor_cd2.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1024, return_dict_in_generate=True, output_scores=True, output_hidden_states=True)
    blackimage_logits = torch.cat(outputs_aug2.scores)  #logits
    blackimage_probs = nn.Softmax(dim=-1)(blackimage_logits)
    blackimage_generated_ids = outputs_aug2.sequences[:, input_ids.shape[-1]:]

    # attention based masking forward pass
    with torch.inference_mode():
        outputs_aug3 = model.generate(input_ids, 
        images=image_tensor_cd3.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1024, return_dict_in_generate=True, output_scores=True)
    attn_logits = torch.cat(outputs_aug3.scores)  #logits
    attn_probs = nn.Softmax(dim=-1)(attn_logits)
    attn_generated_ids = outputs_aug3.sequences[:, input_ids.shape[-1]:]

    # Diffusion Noise
    with torch.inference_mode():
        outputs_aug = model.generate(input_ids, 
        images=image_tensor_cd.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1024, return_dict_in_generate=True, output_scores=True)
    diff_logits = torch.cat(outputs_aug.scores)  #logits
    diff_probs = nn.Softmax(dim=-1)(diff_logits)
    diff_generated_ids = outputs_aug.sequences[:, input_ids.shape[-1]:]

    # Greedy search Original image
    with torch.inference_mode():
        outputs = model.generate(input_ids, 
        images=image_tensor.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1024,  return_dict_in_generate=True, output_scores=True)
    logits = torch.cat(outputs.scores)
    token_probs = nn.Softmax(dim=-1)(logits)
    generated_ids = outputs.sequences[:, input_ids.shape[-1]:]

    flattened_values = []
    for ii in range(token_probs.shape[0]):
        flattened_values.append(float(token_probs[ii][generated_ids[0][ii]].cpu()))

    blackimage_flattened_values = []
    for ii in range(blackimage_probs.shape[0]):
        if generated_ids[0].shape[0] > ii:
            blackimage_flattened_values.append(float(blackimage_probs[ii][generated_ids[0][ii]].cpu()))

    attn_flattened_values = []
    for ii in range(attn_probs.shape[0]):
        if generated_ids[0].shape[0] > ii:
            attn_flattened_values.append(float(attn_probs[ii][generated_ids[0][ii]].cpu()))           

    diff_flattened_values = []
    for ii in range(diff_probs.shape[0]):
        if generated_ids[0].shape[0] > ii:
            diff_flattened_values.append(float(diff_probs[ii][generated_ids[0][ii]].cpu()))           

    # Log prob
    answer, decoded_token, logprobs_dict = get_answer_with_probability( 
            flattened_values, 
            generated_ids[0].tolist(),
            tokenizer
        )

    # Greedy search
    with torch.inference_mode():
        outputs = model.generate(input_ids, 
        images=image_tensor.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        num_beams=5, max_new_tokens=1024, num_return_sequences=5, return_dict_in_generate=True, output_scores=True)
    generated_ids_ = outputs.sequences[:, input_ids.shape[-1]:]
    transition_scores_ = model.compute_transition_scores(
            outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=True
        )
    beam_answers = []
    beam_logprobs = []
    beam_answers.append(answer)
    beam_logprobs.append(logprobs_dict)
    for beam in range(5):
        beam_answer, decoded_tokens, beam_logprobs_dict = get_answer_with_probability_beams(
            transition_scores_[beam].tolist(), 
            generated_ids_[beam].tolist(), 
            tokenizer)
        beam_answers.append(beam_answer)
        beam_logprobs.append(beam_logprobs_dict)

    _ , _ , blackimage_logprobs_dict = get_answer_with_probability( 
            blackimage_flattened_values, 
            generated_ids[0][:blackimage_probs.shape[0]].tolist(),
            tokenizer
        )

    blackimage_answer , _ , _ = get_answer_with_probability( 
            blackimage_flattened_values, 
            blackimage_generated_ids[0][:blackimage_probs.shape[0]].tolist(),
            tokenizer
        )

    _ , _, attn_logprobs_dict = get_answer_with_probability( 
            attn_flattened_values, 
            generated_ids[0][:attn_probs.shape[0]].tolist(),
            tokenizer
        )

    _ , _, diff_logprobs_dict = get_answer_with_probability( 
            diff_flattened_values, 
            generated_ids[0][:diff_probs.shape[0]].tolist(),
            tokenizer
        )

    k = 5
    j = 0
    values, indices5 = torch.topk(token_probs, 5)
    values, indices2 = torch.topk(token_probs, 2)  

    distance_reg_black = calculate_distance(np.array(token_probs[j][indices5[j]].cpu().numpy()), np.array(blackimage_probs[j][indices5[j]].cpu().numpy()), np.array(token_probs[j][indices2[j]].cpu().numpy()), np.array(blackimage_probs[j][indices2[j]].cpu().numpy()), [])
    distance_reg_attn = calculate_distance(np.array(token_probs[j][indices5[j]].cpu().numpy()), np.array(attn_probs[j][indices5[j]].cpu().numpy()), np.array(token_probs[j][indices2[j]].cpu().numpy()), np.array(attn_probs[j][indices2[j]].cpu().numpy()), [])
    distance_reg_diff = calculate_distance(np.array(token_probs[j][indices5[j]].cpu().numpy()), np.array(diff_probs[j][indices5[j]].cpu().numpy()), np.array(token_probs[j][indices2[j]].cpu().numpy()), np.array(diff_probs[j][indices2[j]].cpu().numpy()), [])


    lave_reasoning, lave_score = lave_scorer.compute(
    prediction=answer,
    references=answers_dict,
    question=qs,
    )

    #self eval
    yn_tokens = tokenizer.tokenize("yesYes Yesyesno noNo No")
    yn_token_ids = tokenizer.convert_tokens_to_ids(yn_tokens)
    yes_token_ids = yn_token_ids[:4]
    no_token_ids = yn_token_ids[4:]

    self_prompting_confidence_reg = []
    self_prompting_confidence_aug = []


    cur_prompt = datum['question']
    eval_prompt = "USER: <image> Question: In this image, "+ cur_prompt+ "\nAnswer: " + answer + "\nIs the given answer correct for the question based on the given image? Options: yes, no. ASSISTANT:"
    inputs = tokenizer_image_token(eval_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(device=device, non_blocking=True)

    with torch.inference_mode():
        outputs_ = model.generate(inputs, 
        images=image_tensor.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    logits = outputs_.scores[0]

    token_probs = nn.Softmax(dim=-1)(logits)
    values, indices = torch.topk(token_probs, 15)
    ids = [x for x in indices]

    yes_probs = token_probs[:, yes_token_ids].sum(dim=-1)#.tolist()
    no_probs = token_probs[:, no_token_ids].sum(dim=-1)#.tolist()
    yn_logits_reg = torch.cat([yes_probs.unsqueeze(1), no_probs.unsqueeze(1)], dim=1)
    yn_probs_reg = [[(y/(y+n)).item(), (n/(y+n)).item()] for y, n in zip(yes_probs, no_probs)]
    self_prompting_confidence_reg.append(yn_probs_reg[0][0])
    logprobs_dict["self_prompting_conf_reg"] = yn_probs_reg[0][0]
    logprobs_dict["yn_logits_reg"] = yn_logits_reg[0][0].cpu().numpy().tolist()
    
    with torch.inference_mode():
        outputs_ = model.generate(inputs, 
        images=image_tensor_cd.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    logits = outputs_.scores[0]

    token_probs = nn.Softmax(dim=-1)(logits)
    yn_tokens = tokenizer.tokenize("yesYes Yesyesno noNo No")
    yes_probs = token_probs[:, yes_token_ids].sum(dim=-1)#.tolist()
    no_probs = token_probs[:, no_token_ids].sum(dim=-1)#.tolist()
    yn_logits_aug = torch.cat([yes_probs.unsqueeze(1), no_probs.unsqueeze(1)], dim=1)
    yn_probs_aug = [[(y/(y+n)).item(), (n/(y+n)).item()] for y, n in zip(yes_probs, no_probs)]
    logprobs_dict["self_prompting_diff"] = yn_probs_aug[0][0]
    logprobs_dict["yn_logits_diff"] = yn_logits_aug[0][0].cpu().numpy().tolist()

    with torch.inference_mode():
        outputs_ = model.generate(inputs, 
        images=image_tensor_cd2.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    logits = outputs_.scores[0]

    token_probs = nn.Softmax(dim=-1)(logits)
    yn_tokens = tokenizer.tokenize("yesYes Yesyesno noNo No")
    yes_probs = token_probs[:, yes_token_ids].sum(dim=-1)#.tolist()
    no_probs = token_probs[:, no_token_ids].sum(dim=-1)#.tolist()
    yn_logits_aug = torch.cat([yes_probs.unsqueeze(1), no_probs.unsqueeze(1)], dim=1)
    yn_probs_aug = [[(y/(y+n)).item(), (n/(y+n)).item()] for y, n in zip(yes_probs, no_probs)]
    logprobs_dict["self_prompting_black"] = yn_probs_aug[0][0]
    logprobs_dict["yn_logits_black"] = yn_logits_aug[0][0].cpu().numpy().tolist()
    # self_prompting_confidence_aug.append(yn_probs_aug[0][0])

    with torch.inference_mode():
        outputs_ = model.generate(inputs, 
        images=image_tensor_cd3.unsqueeze(0).to(dtype=torch.float16, device=device, non_blocking=True),
        max_new_tokens=1, return_dict_in_generate=True, output_scores=True)
    logits = outputs_.scores[0]

    token_probs = nn.Softmax(dim=-1)(logits)
    yn_tokens = tokenizer.tokenize("yesYes Yesyesno noNo No")
    yes_probs = token_probs[:, yes_token_ids].sum(dim=-1)#.tolist()
    no_probs = token_probs[:, no_token_ids].sum(dim=-1)#.tolist()
    yn_logits_aug = torch.cat([yes_probs.unsqueeze(1), no_probs.unsqueeze(1)], dim=1)
    yn_probs_aug = [[(y/(y+n)).item(), (n/(y+n)).item()] for y, n in zip(yes_probs, no_probs)]
    logprobs_dict["self_prompting_attn"] = yn_probs_aug[0][0]
    logprobs_dict["yn_logits_attn"] = yn_logits_aug[0][0].cpu().numpy().tolist()

    directvqa_rollouts[i] = {
        'qid': i,
        'image_id': datum['image_id'],
        'vqa_question': qs, 
        'annotated_answers': answers_dict,
        'answer': answer, 
        'blackimage_answer': blackimage_answer,
        'beam_answers': beam_answers,
        'beam_logprobs': beam_logprobs,
        'answer_logprobs_dict': logprobs_dict,
        'blackimage_logprobs_dict': blackimage_logprobs_dict,
        'attn_logprobs_dict': attn_logprobs_dict,
        'diff_logprobs_dict': diff_logprobs_dict,
        'distance_reg_black': distance_reg_black,
        'distance_reg_attn': distance_reg_attn,
        'distance_reg_diff': distance_reg_diff,
        # 'score': float(score), 
        'lave_score': float(lave_score),
        'lave_reasoning': lave_reasoning,
    }
    json.dump(directvqa_rollouts, open(answers_file, 'w'), indent=2)





1075


1075it [52:12,  2.91s/it]
